In [ ]:
import pandas as pd

In [ ]:
!git clone https://github.com/tdeme/NLP-Political-Bias.git

In [ ]:
#Since data is already scrapped, import files
left_tweets_df = pd.read_csv('/content/NLP-Political-Bias/Rleft_tweets.csv')
right_tweets_df = pd.read_csv('/content/NLP-Political-Bias/Rright_tweets.csv')

In [ ]:
print(len(left_tweets_df))
print(len(right_tweets_df))

49532
40105


In [ ]:
#Next will train a transformer neural net
#To do this, make lists for our data:

left_tweet_list = left_tweets_df['tweet'].to_list()
left_labels = [0]*len(left_tweets_df['tweet'].to_list())

right_tweet_list = right_tweets_df['tweet'].to_list()
right_labels = [1]*len(right_tweets_df['tweet'].to_list())

train_tweets = left_tweet_list+right_tweet_list
train_labels = left_labels+right_labels

In [ ]:
from sklearn.model_selection import train_test_split
train_texts, val_texts, train_labels, val_labels = train_test_split(train_tweets, train_labels, test_size=.2, random_state = 42)

In [ ]:
len(val_texts)

17928

In [ ]:
!pip install transformers
from transformers import DistilBertTokenizerFast
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)
test_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
import torch

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)
test_dataset = TweetDataset(test_encodings, val_labels)

In [ ]:
!pip install datasets
import numpy as np
from datasets import load_metric

metric = load_metric("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=3,              # total number of training epochs
    per_device_train_batch_size=16,  # batch size per device during training
    per_device_eval_batch_size=64,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    evaluation_strategy='epoch'
)

model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")

trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,            # evaluation dataset
    compute_metrics=compute_metrics             
)

trainer.train()

In [ ]:
#Finally, save the model to the specified directory
save_directory = './save_model'
tokenizer.save_pretrained(save_directory)
model.save_pretrained(save_directory)